In [ ]:
library(Seurat)
library(tidyverse)
library(Matrix)
library(RColorBrewer)
library(scatterpie)

In [ ]:
meta <- read.csv("/sciSpace/GSE166692_sciSpace_cell_metadata.tsv.gz",header=T,stringsAsFactors=F,sep="\t")
genes <- read.csv("/sciSpace/GSE166692_sciSpace_gene_metadata.tsv.gz",header=T,stringsAsFactors=F,sep="\t")

In [ ]:
# The following RDS files are products from import_DGE.R
seu_CR <- readRDS("intermediate_data/sciSpace/exp2_SeuratObj.rds")
seu_orig <- readRDS("exp2_origSciSpace_SeuratObj.rds")

In [ ]:
df <- read.table("barcode_correspondence_data/cb_10x_to_sci.txt",sep="\t",header=F)
conv_v <- df$V2 %>% as.character
names(conv_v) <- df$V1 %>% as.character

In [ ]:
seu_CR %>% colnames %>% head
seu_orig %>% colnames %>% head

In [ ]:
seu_orig$seurat_clusters %>% head

In [ ]:
qual_col_pals = brewer.pal.info[brewer.pal.info$category == 'qual',]
col_vector = unlist(mapply(brewer.pal, qual_col_pals$maxcolors, rownames(qual_col_pals)))

set.seed(12)
my_col_viri <- sample(col_vector, length(unique(seu_CR$seurat_clusters)))
p1 <- DimPlot(seu_CR,cols = my_col_viri) + theme_void() + NoLegend()

set.seed(3)
my_col_viri2 <- sample(col_vector, length(unique(seu_orig$seurat_clusters)))
p2 <- DimPlot(seu_orig,cols = my_col_viri2) + theme_void()+ NoLegend()

In [ ]:
ggsave("translated_umap.jpeg",p1,width = 12,height = 12,units = "cm",dpi = 300)

In [ ]:
ggsave("orig_umap.jpeg",p2,width = 12,height = 12,units = "cm",dpi = 300)

In [ ]:
seu_CR_meta <- seu_CR@meta.data
seu_CR_meta$CellID <- rownames(seu_CR_meta)

In [ ]:
sci_CR_meta_merge <- merge(meta,seu_CR_meta,by.x = 1,by.y=9)

In [ ]:
slide="Slide 14"
df.slide.select <- filter(sci_CR_meta_merge,slide_id==slide)

df.shrink <- df.slide.select[,c("Cell","Row","Col","final_cluster_label","seurat_clusters")]
df.shrink$RowCol <- paste0(df.shrink$Row,"_",df.shrink$Col)
df.shrink.aggr <- aggregate(df.shrink$seurat_clusters,list(df.shrink$RowCol),table)
df.shrink.aggr <- df.shrink.aggr %>% as.matrix() %>% as.data.frame
df.shrink.coord <- str_split_fixed(string = df.shrink.aggr$Group.1,n=2,pattern = "_") %>% as.data.frame() %>% apply(c(1,2),as.integer) %>% as.data.frame()
df.shrink.coord$rowcol <- df.shrink.aggr$Group.1
df.shrink.ratio <- df.shrink.aggr[,-1] %>% apply(c(1,2),as.integer) %>% as.data.frame
df.shrink.ratio <- df.shrink.ratio / rowSums(df.shrink.ratio)
colnames(df.shrink.coord) <- c("row","col","rowcol")
colnames(df.shrink.ratio) <- sub("x.","",colnames(df.shrink.ratio))
df.shrink.merge <- cbind(df.shrink.coord,df.shrink.ratio)

g <- ggplot()+
  geom_scatterpie(data = df.shrink.merge,aes(x=col,y=row,group=rowcol),cols = colnames(df.shrink.merge)[4:ncol(df.shrink.merge)],color=NA,pie_scale=.5)+
  theme_classic()+
  coord_equal()+
  scale_fill_manual(values = my_col_viri)+
theme_void()+
NoLegend()+
  theme(rect = element_rect(fill = "transparent"))

In [ ]:
ggsave("spatial_pie_10x.png",g,width = 14,height = 12,units = "cm",dpi = 400)

In [ ]:
seu_orig_meta <- seu_orig@meta.data
seu_orig_meta$CellID <- rownames(seu_orig_meta)

In [ ]:
sci_orig_meta_merge <- merge(meta,seu_orig_meta,by.x = 1,by.y=9)

In [ ]:
slide="Slide 14"
df.slide.select <- filter(sci_orig_meta_merge,slide_id==slide)

df.shrink <- df.slide.select[,c("Cell","Row","Col","final_cluster_label","seurat_clusters")]
df.shrink$RowCol <- paste0(df.shrink$Row,"_",df.shrink$Col)
# df.shrink.aggr <- aggregate(df.shrink$final_cluster_label,list(df.shrink$RowCol),table)
df.shrink.aggr <- aggregate(df.shrink$seurat_clusters,list(df.shrink$RowCol),table)
df.shrink.aggr <- df.shrink.aggr %>% as.matrix() %>% as.data.frame
df.shrink.coord <- str_split_fixed(string = df.shrink.aggr$Group.1,n=2,pattern = "_") %>% as.data.frame() %>% apply(c(1,2),as.integer) %>% as.data.frame()
df.shrink.coord$rowcol <- df.shrink.aggr$Group.1
df.shrink.ratio <- df.shrink.aggr[,-1] %>% apply(c(1,2),as.integer) %>% as.data.frame
df.shrink.ratio <- df.shrink.ratio / rowSums(df.shrink.ratio)
colnames(df.shrink.coord) <- c("row","col","rowcol")
colnames(df.shrink.ratio) <- sub("x.","",colnames(df.shrink.ratio))
df.shrink.merge <- cbind(df.shrink.coord,df.shrink.ratio)

g <- ggplot()+
  geom_scatterpie(data = df.shrink.merge,aes(x=col,y=row,group=rowcol),cols = colnames(df.shrink.merge)[4:ncol(df.shrink.merge)],color=NA,pie_scale=.5)+
  theme_classic()+
  coord_equal()+
  scale_fill_manual(values = my_col_viri2)+
theme_void()+
NoLegend()+
  theme(rect = element_rect(fill = "transparent"))

In [ ]:
ggsave("spatial_pie_orig.png",g,width = 14,height = 12,units = "cm",dpi = 400)